# Online Clustering of Bandits
Apprentissage en ligne et aggrégation

Antoine GRELETY, Vincent LE MEUR

L'object de ce notebook est de résumer l'article suivant : https://arxiv.org/pdf/1401.8257.pdf et d'en proposer une implémentation en python.

<b> Attention les cellules commençant par "TL" sont issues d'une tradition litérale à retravailler </b>

## Introduction

Enjeu : Incorporer des relations type réseaux/graphs dans des algorithmes du bandit (typiquement pour de la recommandation) avec un coût computationnel raisonnable. 

Plusieurs approches ont été réalisées mais qui ont des inconvénients concernant la mise en pratique :
- L'information des réseaux sociaux peut être trompeuse ou inexistante
- Les algorithmes utilisés sont peu ou pas scalables

Il est par ailleurs possible dans ces réseaux  d'identifier des groupes "homogènes" d'utilisateurs. Cela permet d'effectuer des recommandations par groupe et de ne pas avoir à entraîner un modèle par utilisateur

Plan de l'article : 
- Une étude théorique et empirique d'un algorithme de "clustering adaptatif" pour "linear bandits (on cherche à toucher n utilisateurs organisé selon m groupes (m<<n)
- Mise en place de l'algo et Analyse du regret en $\mathrm{0}(\sqrt{T})$ (le grand oh dépendant de m (pas de n))
- Mise en pratique sur des datasets réels

TL : 
L'idée principale de l'algorithme est d'utiliser "des balles de confiance" des modèles des utilisateurs à la fois pour estimer la similarité des utilisateurs et pour partager les commentaires entre utilisateurs (réputés similaires).

L'algorithme est de plus adaptatif entre une situation avec une recommandation pour tous les utilisateurs et une recommandation personnalisé pour chaque utilisateur.

## Modèle d'apprentissage

<b>Hypothèse fondamentale</b> : On suppose qu'il existe un bon clustering permettant de prendre en compte l'information de similarité entre individus.

Ainsi, soit $V = \{1,..n\}$, l'ensemble des utilisateurs et $V_1,...,V_m$ les clusters associés. L'hypothèse précedente considère que dans chacun de ces clusters, les utilisateurs ont le même comportement.

La partition de V (nombre de clusters et nombre d'individus par clusters) est inconnu et doit être inféré par l'algorithme au fil de l'eau.

L'apprentissage est séquentiel : 
- A chaque tour on reçoit un id $i_t \in V$ tiré uniformément et un "vecteur contexte" $C_{i_t}=\{x_{t,1},...,x_{t,c_t}\}$
- On sélectionne alors des $x_{t,k_t}^{-} \in C_{i_t}$ à recommander à l'utilisateur et on calcule une fonction $a_t(x_{t,k_t},i_t) \in \mathcal{R}$ de paiemment

Le nombre $c_t$ de vecteurs contexte est fonction des précédents $a_t$,$C_{i_t}$ et $i_t$.

Ainsi que la séquence $C_{i_t}=\{x_{t,1},...,x_{t,c_t}\}$ est générée de manière iid à partir d'un processus aléatoire pour lequel la matrice $E(XX^T)$ est "full rank"

Enfin, les paiemments sont de la forme suivante : $a_i(x) = u_{j(i)}^{T}x + \epsilon_{j(i)}(x)$ où  $u_{j}$ représente un vecteur paramètre inconnu associé à chaque cluster $V_j$ auquel appartient l'utilisateur $i$ et $\epsilon_{j(i)}$ représente un terme de variance conditionnelle bruitée centrée et de variance $\sigma$ finie. "Conditionnelle" car calculé à partir d'une espérance conditionnelle par rapport à toutes les précédentes évaluations de $C_{i_t},i_t,a_i$.

Il y a également plusieurs hypothèses fortes : 
- $a_i(x) \in [-1,1]$
- Pour tout $j \neq j'$, $||u_j - u_{j'}|| >= \gamma$ (hypothèses d'hétéorogénité entre clusters

On définit alors le regret à la date t par : 
$r_t = (max_{x \in C_{it}}u_{j(i(t))}^{T}x) - u_{j(i(t))}^{T}x^{—}$

L'un des objectif principaux est de borner le risque cumulé défini par $\sum_{t=1}^T (r_{t})$

On peut le border par : $\sum_{t=1}^T (r_{t}) = O_{log}(\sum_{j=1}^m(\sigma d + ||u_j||\sqrt{d})\sqrt{T}$

## L'algorithme CLUB

En réalité plutôt que d'avoir un $u_j$ par cluster on a n $u_j$ avec égalité entre ces vecteurs si ils appartiennent au même cluster.

A chaque temps $t$ l'algorithme aura une estimation des moindres carrées $w_{i,t}$ de $u_i$.
$w_{i,t}$ se décompose par le produit suivant : $M_{i,t}^{-1}\times b_{i,t}$

L'algorithme mets à jour de manière séquentielle la partition de V en clusters.

A chaque date t, l'algorithme reçoit un utilisateur $i_{t}$ ainsi que son vecteur contexte associé. L'algorithme sélectionne une composante de ce vecteur à recommander à l'utilisateur $i_{t}$ en identifiant le cluster auquel appartient $i_{t}$ et en updatant les différents paramètres.

## Notre implémentation

Le pseudo code à implémenter est le suivant : 

![title](algo.png)

<b>Problème : Structure du Graph</b>